In [3]:
##this makes the i95 master file and then below pulls all the dat afrom the TMAS files and cleans it and adds noise estimation based on vehicle mix 

import pandas as pd
from pathlib import Path

# Configuration
DATA_DIR = "/Users/lauralubben/repos/traffic/2023_station_data"
OUTPUT_FILE = "/Users/lauralubben/repos/traffic/Cleaning/I95_stations_master.csv"

# States to process
STATES = ['DC', 'MD', 'VA', 'NJ', 'PA', 'DE', 'VA']

def process_i95_stations():
    all_stations = []
    
    for state in STATES:
        file_path = Path(DATA_DIR) / f"{state}_2023 (TMAS).STA"
        
        if file_path.exists():
            print(f"Processing {state}...")
            
            # Read the station file
            df = pd.read_csv(file_path, sep='|', header=0, dtype=str)
            
            # Clean up column names
            df.columns = df.columns.str.strip()
            
            # Fix coordinates
            df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce') / 1000000
            df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce') / 1000000
            
            # Fix longitude for US (should be negative)
            df['longitude'] = -abs(df['longitude'])
            
            # Filter for stations with "95" in posted_signed_route
            i95_stations = df[df['posted_signed_route'].str.contains('95', na=False)]
            
            # Add state column
            i95_stations['state'] = state
            
            print(f"  Found {len(i95_stations)} I-95 stations in {state}")
            
            if len(i95_stations) > 0:
                all_stations.append(i95_stations)
        else:
            print(f"File not found: {file_path}")
    
    if all_stations:
        # Combine all states
        combined_stations = pd.concat(all_stations, ignore_index=True)
        
        # Save to CSV
        combined_stations.to_csv(OUTPUT_FILE, index=False)
        
        print(f"\nSaved {len(combined_stations)} I-95 stations to {OUTPUT_FILE}")
        print(f"States included: {combined_stations['state_code'].value_counts().to_dict()}")
        
        return combined_stations
    else:
        print("No stations found")
        return pd.DataFrame()

# Run the script
if __name__ == "__main__":
    stations = process_i95_stations()

Processing DC...
  Found 17 I-95 stations in DC
Processing MD...
  Found 152 I-95 stations in MD
Processing VA...
  Found 275 I-95 stations in VA
Processing NJ...
  Found 135 I-95 stations in NJ
Processing PA...
  Found 14 I-95 stations in PA
Processing DE...
  Found 199 I-95 stations in DE
Processing VA...
  Found 275 I-95 stations in VA

Saved 1067 I-95 stations to /Users/lauralubben/repos/traffic/Cleaning/I95_stations_master.csv
States included: {'51': 550, '10': 199, '24': 152, '34': 135, '11': 17, '42': 14}


/var/folders/xq/6c06whd96gng5c42_wr6kslr0000gq/T/ipykernel_63232/300984706.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i95_stations['state'] = state
/var/folders/xq/6c06whd96gng5c42_wr6kslr0000gq/T/ipykernel_63232/300984706.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  i95_stations['state'] = state
/var/folders/xq/6c06whd96gng5c42_wr6kslr0000gq/T/ipykernel_63232/300984706.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [5]:
import pandas as pd
import zipfile
import tempfile
import os
from pathlib import Path

# ============================================================================
# FILE PATHS - UPDATE THESE FOR YOUR SETUP
# ============================================================================

# Path to your I-95 stations master file
STATIONS_FILE = "/Users/lauralubben/repos/traffic/Cleaning/I95_stations_master.csv"

# Directory containing your downloaded TMAS zip files
INPUT_DIR = "/Users/lauralubben/repos/traffic/Cleaning/raw_tmas_data"

# Directory to save processed files
OUTPUT_DIR = "/Users/lauralubben/repos/traffic/Cleaning/processed_files"

# Your TMAS zip files (March-June 2023)
TMAS_ZIP_FILES = [
    "mar_2020_ccs_data.zip",
    "apr_2020_ccs_data.zip", 
    "may_2020_ccs_data.zip",
    "jun_2020_ccs_data.zip",

    # 2021
    "mar_2021_ccs_data.zip",
    "apr_2021_ccs_data.zip", 
    "may_2021_ccs_data.zip",
    "jun_2021_ccs_data.zip",

    # 2022
    "mar_2022_ccs_data.zip",
    "apr_2022_ccs_data.zip", 
    "may_2022_ccs_data.zip",
    "jun_2022_ccs_data.zip",
    
    "mar_2023_ccs_data.zip",
    "apr_2023_ccs_data.zip", 
    "may_2023_ccs_data.zip",
    "jun_2023_ccs_data.zip"
]

# Final combined output file name
FINAL_OUTPUT_FILE = "i95_traffic_mar_jun_2020_2023.csv"

# ============================================================================
class TmasI95Processor:
    def __init__(self, stations_file):
        """
        Initialize with your I-95 stations master file
        """
        self.stations_df = pd.read_csv(stations_file)
        # Create lookup set for fast filtering
        self.i95_stations = set(
            zip(self.stations_df['state_code'], 
                self.stations_df['station_id'].astype(str))
        )
        
        # Create lookup dict for station metadata
        self.station_metadata = {}
        for _, row in self.stations_df.iterrows():
            key = (row['state_code'], str(row['station_id']))
            self.station_metadata[key] = {
                'latitude': row['latitude'],
                'longitude': row['longitude'], 
                'station_location': row['station_location']
            }
        
        print(f"Loaded {len(self.i95_stations)} I-95 stations with metadata")
        
    def process_tmas_zip(self, zip_file_path, output_file=None):
        """
        Process a TMAS zip file and extract only I-95 stations in wide format
        """
        print(f"Processing: {zip_file_path}")
        
        # Create temp directory for extraction
        with tempfile.TemporaryDirectory() as temp_dir:
            # Extract zip file
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(temp_dir)
            
            # Find VOL files for I-95 states only (DC, MD, VA, NJ, PA, DE)
            i95_state_prefixes = ['DC_', 'MD_', 'VA_', 'NJ_', 'PA_', 'DE_']
            
            vol_files = []
            for prefix in i95_state_prefixes:
                matching_files = list(Path(temp_dir).glob(f'**/{prefix}*.VOL'))
                vol_files.extend(matching_files)
            
            if not vol_files:
                print(f"No I-95 state VOL files found in {zip_file_path}")
                return None
            
            all_data = []
            
            for vol_file in vol_files:
                print(f"  Processing: {vol_file.name}")
                
                # Read file and detect format
                try:
                    # First, peek at the file to detect format
                    with open(vol_file, 'r') as f:
                        first_line = f.readline().strip()
                        second_line = f.readline().strip()
                    
                    if '|' in first_line:
                        # Check if first line contains headers or data
                        if 'record_type' in first_line.lower():
                            # Pipe-delimited with headers (2022-2023)
                            print(f"    Detected pipe-delimited format with headers")
                            df = pd.read_csv(vol_file, sep='|', dtype={'station_id': str})
                        else:
                            # Pipe-delimited without headers (2021)
                            print(f"    Detected pipe-delimited format without headers")
                            df = self.parse_pipe_no_headers(vol_file)
                            
                    else:
                        # Fixed-width format (2020)
                        print(f"    Detected fixed-width format")
                        df = self.parse_fixed_width_tmas(vol_file)
                        
                    if df is None:
                        continue
                    
                    # Filter to I-95 stations by doing an inner merge with master stations
                    stations_subset = self.stations_df[['state_code', 'station_id', 'latitude', 'longitude', 'station_location', 'state']].copy()
                    stations_subset['station_id'] = stations_subset['station_id'].astype(str)
                    
                    df_filtered = df.merge(
                        stations_subset,
                        on=['state_code', 'station_id'],
                        how='inner'  # Only keep stations that exist in master file
                    )
                    
                    if len(df_filtered) == 0:
                        print(f"    No I-95 stations found in {vol_file.name}")
                        continue
                    
                    print(f"    Found {len(df_filtered)} I-95 station-day records")
                    
                    # Clean the data but keep wide format
                    cleaned_data = self.clean_wide_format(df_filtered)
                    all_data.append(cleaned_data)
                    
                except Exception as e:
                    print(f"    Error processing {vol_file.name}: {e}")
                    continue
            
            if not all_data:
                print("No data processed successfully")
                return None
            
            # Combine all data
            final_data = pd.concat(all_data, ignore_index=True)
            
            # Remove duplicates (keep first occurrence)
            initial_count = len(final_data)
            final_data = final_data.drop_duplicates(
                subset=['state_code', 'station_id', 'travel_dir', 'travel_lane', 
                       'year_record', 'month_record', 'day_record'],
                keep='first'
            )
            final_count = len(final_data)
            
            if initial_count > final_count:
                print(f"    Removed {initial_count - final_count} duplicate records")
            
            # Generate output filename if not provided
            if output_file is None:
                zip_name = Path(zip_file_path).stem
                output_file = f"i95_{zip_name}_wide.csv"
            
            # Save to CSV
            final_data.to_csv(output_file, index=False)
            print(f"Saved {len(final_data)} station-day records to: {output_file}")
            
            return final_data
    
    def clean_wide_format(self, df):
        """
        Clean the data, keep wide format, and calculate noise metrics
        """
        # Clean up numeric hour columns
        hour_cols = [f'hour_{i:02d}' for i in range(24)]
        
        # Check which hour columns exist
        existing_hour_cols = [col for col in hour_cols if col in df.columns]
        
        # Convert hour columns to numeric, handling any formatting issues
        for col in existing_hour_cols:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Calculate noise metrics
        df = self.calculate_noise_metrics(df, existing_hour_cols)
        
        # Sort by station, date for clean output
        df_sorted = df.sort_values([
            'state_code', 'station_id', 'year_record', 'month_record', 'day_record'
        ]).reset_index(drop=True)
        
        return df_sorted
    
    def parse_fixed_width_tmas(self, file_path):
        """
        Parse fixed-width TMAS format (2020-2021) into DataFrame
        Based on TMAS 2001 format specifications
        """
        try:
            # Column specifications for fixed-width format
            column_names = ['record_type','state_code','f_system','station_id','travel_dir','travel_lane',
                           'year_record','month_record','day_record','day_of_week',
                           'hour_00','hour_01','hour_02','hour_03','hour_04','hour_05','hour_06','hour_07',
                           'hour_08','hour_09','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15',
                           'hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23',
                           'restrictions']
            
            colspecs = [(0,1),(1,3),(3,5),(5,11),(11,12),(12,13),(13,15),(15,17),(17,19),(19,20),
                       (20,25),(25,30),(30,35),(35,40),(40,45),(45,50),(50,55),(55,60),(60,65),(65,70),
                       (70,75),(75,80),(80,85),(85,90),(90,95),(95,100),(100,105),(105,110),(110,115),
                       (115,120),(120,125),(125,130),(130,135),(135,140),(140,141)]
            
            # Read fixed-width file
            df = pd.read_fwf(file_path, names=column_names, colspecs=colspecs, dtype=str)
            
            # Convert numeric columns
            numeric_cols = ['state_code', 'travel_dir', 'travel_lane', 'year_record', 
                           'month_record', 'day_record', 'day_of_week'] + \
                          [f'hour_{i:02d}' for i in range(24)]
            
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Convert 2-digit year to 4-digit (e.g., 20 -> 2020)
            df['year_record'] = 2000 + df['year_record']
            
            return df
            
        except Exception as e:
            print(f"    Error parsing fixed-width format: {e}")
            return None
    
    def parse_pipe_no_headers(self, file_path):
        """
        Parse pipe-delimited TMAS format without headers (2021)
        """
        try:
            # Column names for pipe-delimited format
            column_names = ['record_type','state_code','f_system','station_id','travel_dir','travel_lane',
                           'year_record','month_record','day_record','day_of_week',
                           'hour_00','hour_01','hour_02','hour_03','hour_04','hour_05','hour_06','hour_07',
                           'hour_08','hour_09','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15',
                           'hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23',
                           'restrictions']
            
            # Read pipe-delimited file without headers
            df = pd.read_csv(file_path, sep='|', names=column_names, dtype={'station_id': str})
            
            # Convert numeric columns  
            numeric_cols = ['state_code', 'travel_dir', 'travel_lane', 'year_record', 
                           'month_record', 'day_record', 'day_of_week'] + \
                          [f'hour_{i:02d}' for i in range(24)]
            
            for col in numeric_cols:
                if col in df.columns:
                    df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Convert 2-digit year to 4-digit (e.g., 21 -> 2021)
            df['year_record'] = 2000 + df['year_record']
            
            return df
            
        except Exception as e:
            print(f"    Error parsing pipe format without headers: {e}")
            return None
    
    def calculate_noise_metrics(self, df, hour_cols):
        """
        Calculate daily_avg_noise, peak_hour_noise, and overnight_noise from hourly volumes
        Uses your formula: 70+10*LOG10(volume/100000)+(cars%/100)*0+(trucks%/100)*8+(suvs%/100)*3
        
        Vehicle mix by year (trucks steady at 10%, SUVs increasing, cars decreasing):
        - 2020: Cars 60%, Trucks 10%, SUVs 30%
        - 2021: Cars 58%, Trucks 10%, SUVs 32% 
        - 2022: Cars 56%, Trucks 10%, SUVs 34%
        - 2023: Cars 54%, Trucks 10%, SUVs 36%
        """
        import numpy as np
        
        # Get vehicle mix based on year (reflecting real SUV dominance)
        def get_vehicle_mix(year):
            if year == 2020:
                return {'cars': 45, 'trucks': 10, 'suvs': 45}  # SUVs already dominant
            elif year == 2021:
                return {'cars': 42, 'trucks': 10, 'suvs': 48}  # Continuing SUV growth
            elif year == 2022:
                return {'cars': 39, 'trucks': 10, 'suvs': 51}  # SUVs become majority
            elif year == 2023:
                return {'cars': 36, 'trucks': 10, 'suvs': 54}  # SUV dominance increases
            else:
                # Default for any other year
                return {'cars': 45, 'trucks': 10, 'suvs': 45}
        
        # Calculate vehicle mix adjustment for each row based on year
        df['vehicle_mix_adjustment'] = df['year_record'].apply(
            lambda year: (get_vehicle_mix(year)['cars']/100)*0 + 
                        (get_vehicle_mix(year)['trucks']/100)*8 + 
                        (get_vehicle_mix(year)['suvs']/100)*3
        )
        
        # Calculate hourly noise levels using your formula
        for col in hour_cols:
            hour_num = col.split('_')[1]
            noise_col = f'noise_{hour_num}'
            
            # Fixed formula for realistic highway noise levels
            # Base formula: 55 + 10*LOG10(volume) + vehicle_mix_adjustment
            # This gives realistic ranges: ~73-92 dB for volumes 70-5000
            df[noise_col] = np.where(
                (df[col].isna()) | (df[col] <= 0),
                np.nan,  # Handle zero/null volumes
                55 + 10 * np.log10(df[col]) + df['vehicle_mix_adjustment']
            )
        
        # Get noise columns
        noise_cols = [f'noise_{i:02d}' for i in range(24) if f'noise_{i:02d}' in df.columns]
        
        # Calculate summary metrics (ignoring NaN values)
        df['daily_avg_noise'] = df[noise_cols].mean(axis=1, skipna=True)
        df['peak_hour_noise'] = df[noise_cols].max(axis=1, skipna=True)
        
        # Overnight noise (10 PM to 6 AM): hours 22, 23, 00, 01, 02, 03, 04, 05
        overnight_cols = ['noise_22', 'noise_23', 'noise_00', 'noise_01', 
                         'noise_02', 'noise_03', 'noise_04', 'noise_05']
        overnight_cols = [col for col in overnight_cols if col in df.columns]
        df['overnight_noise'] = df[overnight_cols].mean(axis=1, skipna=True)
        
        # Add some additional useful metrics
        # Rush hour noise (7-9 AM and 5-7 PM)
        rush_morning = ['noise_07', 'noise_08', 'noise_09']
        rush_evening = ['noise_17', 'noise_18', 'noise_19']
        rush_cols = [col for col in rush_morning + rush_evening if col in df.columns]
        if rush_cols:
            df['rush_hour_noise'] = df[rush_cols].mean(axis=1, skipna=True)
        
        # Total daily volume (useful for correlation checks)
        df['total_daily_volume'] = df[[col for col in hour_cols if col in df.columns]].sum(axis=1, skipna=True)
        
        # Drop individual hourly noise columns and temp vehicle mix column
        df = df.drop(columns=noise_cols + ['vehicle_mix_adjustment'])
        
        return df
        # Overnight noise (10 PM to 6 AM): hours 22, 23, 00, 01, 02, 03, 04, 05
        overnight_cols = ['noise_22', 'noise_23', 'noise_00', 'noise_01', 
                         'noise_02', 'noise_03', 'noise_04', 'noise_05']
        overnight_cols = [col for col in overnight_cols if col in df.columns]
        df['overnight_noise'] = df[overnight_cols].mean(axis=1)
        
        # Add some additional useful metrics
        # Rush hour noise (7-9 AM and 5-7 PM)
        rush_morning = ['noise_07', 'noise_08', 'noise_09']
        rush_evening = ['noise_17', 'noise_18', 'noise_19']
        rush_cols = [col for col in rush_morning + rush_evening if col in df.columns]
        if rush_cols:
            df['rush_hour_noise'] = df[rush_cols].mean(axis=1)
        
        # Total daily volume (useful for correlation checks)
        df['total_daily_volume'] = df[[col for col in hour_cols if col in df.columns]].sum(axis=1)
        
        # Drop individual hourly noise columns to keep file manageable
        df = df.drop(columns=noise_cols)
        
        return df
    
    def process_multiple_files(self, zip_files, output_dir="processed_data"):
        """
        Process multiple TMAS zip files
        """
        Path(output_dir).mkdir(exist_ok=True)
        
        all_results = []
        
        for zip_file in zip_files:
            output_file = Path(output_dir) / f"i95_{Path(zip_file).stem}_hourly.csv"
            result = self.process_tmas_zip(zip_file, output_file)
            if result is not None:
                all_results.append(result)
        
        return all_results
    
    def combine_monthly_files(self, processed_files, output_file="i95_combined_2023_wide.csv"):
        """
        Combine multiple processed monthly files into one master file
        """
        print(f"Combining {len(processed_files)} files...")
        
        all_data = []
        for file in processed_files:
            df = pd.read_csv(file)
            all_data.append(df)
        
        combined = pd.concat(all_data, ignore_index=True)
        
        # Sort by station, date
        combined = combined.sort_values([
            'state_code', 'station_id', 'year_record', 'month_record', 'day_record'
        ]).reset_index(drop=True)
        
        combined.to_csv(output_file, index=False)
        print(f"Combined file saved: {output_file} ({len(combined)} station-day records)")
        
        return combined

# Example usage:
if __name__ == "__main__":
    # Initialize processor with your stations file
    processor = TmasI95Processor(STATIONS_FILE)
    
    # Create output directory
    Path(OUTPUT_DIR).mkdir(exist_ok=True)
    
    # Process all TMAS zip files
    print("Processing all TMAS files...")
    processed_files = []
    
    for zip_file in TMAS_ZIP_FILES:
        input_path = Path(INPUT_DIR) / zip_file
        output_filename = f"i95_{Path(zip_file).stem}_wide.csv"
        output_path = Path(OUTPUT_DIR) / output_filename
        
        if input_path.exists():
            result = processor.process_tmas_zip(str(input_path), str(output_path))
            if result is not None:
                processed_files.append(str(output_path))
        else:
            print(f"Warning: File not found: {input_path}")
    
    # Combine all processed files into one master file
    if processed_files:
        print(f"\nCombining {len(processed_files)} processed files...")
        final_output_path = Path(OUTPUT_DIR) / FINAL_OUTPUT_FILE
        combined = processor.combine_monthly_files(processed_files, str(final_output_path))
        print(f"\n🎉 Processing complete! Final file: {final_output_path}")
        print(f"   Total station-day records: {len(combined)}")
        print(f"   Unique stations: {combined['station_id'].nunique()}")
        print(f"   Date range: {combined['month_record'].min()}-{combined['month_record'].max()}/2023")
    else:
        print("No files were processed successfully.")

Loaded 364 I-95 stations with metadata
Processing all TMAS files...
Processing: /Users/lauralubben/repos/traffic/Cleaning/raw_tmas_data/mar_2020_ccs_data.zip
  Processing: DC_MAR_2020 (TMAS).VOL
    Detected fixed-width format
    Found 651 I-95 station-day records
  Processing: MD_MAR_2020 (TMAS).VOL
    Detected fixed-width format


/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


    Found 17564 I-95 station-day records
  Processing: VA_MAR_2020 (TMAS).VOL
    Detected fixed-width format
    Found 16378 I-95 station-day records
  Processing: NJ_MAR_2020 (TMAS).VOL
    Detected fixed-width format
    Found 4737 I-95 station-day records


/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lau

  Processing: PA_MAR_2020 (TMAS).VOL
    Detected fixed-width format
    Found 992 I-95 station-day records
  Processing: DE_MAR_2020 (TMAS).VOL
    Detected fixed-width format
    Found 5402 I-95 station-day records
    Removed 34832 duplicate records


...

Saved 11978 station-day records to: /Users/lauralubben/repos/traffic/Cleaning/processed_files/i95_apr_2023_ccs_data_wide.csv
Processing: /Users/lauralubben/repos/traffic/Cleaning/raw_tmas_data/may_2023_ccs_data.zip
  Processing: DC_MAY_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 457 I-95 station-day records
  Processing: MD_MAY_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 13270 I-95 station-day records
  Processing: VA_MAY_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 13766 I-95 station-day records


/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lau

  Processing: NJ_MAY_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 6398 I-95 station-day records
  Processing: PA_MAY_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 2056 I-95 station-day records
  Processing: DE_MAY_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 14271 I-95 station-day records
    Removed 37874 duplicate records
Saved 12344 station-day records to: /Users/lauralubben/repos/traffic/Cleaning/processed_files/i95_may_2023_ccs_data_wide.csv
Processing: /Users/lauralubben/repos/traffic/Cleaning/raw_tmas_data/jun_2023_ccs_data.zip
  Processing: DC_JUN_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 420 I-95 station-day records
  Processing: MD_JUN_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 14081 I-95 station-day records
  Processing: VA_JUN_2023 (TMAS).VOL
    Detected pipe-delimited format with headers


/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lauralubben/Library/Python/3.9/lib/python/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/lau

    Found 13114 I-95 station-day records
  Processing: NJ_JUN_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 5310 I-95 station-day records
  Processing: PA_JUN_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 2004 I-95 station-day records
  Processing: DE_JUN_2023 (TMAS).VOL
    Detected pipe-delimited format with headers
    Found 14580 I-95 station-day records
    Removed 37431 duplicate records
Saved 12078 station-day records to: /Users/lauralubben/repos/traffic/Cleaning/processed_files/i95_jun_2023_ccs_data_wide.csv

Combining 15 processed files...
Combining 15 files...
Combined file saved: /Users/lauralubben/repos/traffic/Cleaning/processed_files/i95_traffic_mar_jun_2020_2023.csv (172685 station-day records)

🎉 Processing complete! Final file: /Users/lauralubben/repos/traffic/Cleaning/processed_files/i95_traffic_mar_jun_2020_2023.csv
   Total station-day records: 172685
   Unique stations: 294
   Date range: 3-6/2023
